# MODULE CS2230 - CÁC MÔ HÌNH HỌC SÂU VÀ ỨNG DỤNG

**Hướng dẫn**:
- Khởi động kernel: Ở thanh menu, chọn Kernel$\rightarrow$Restart kernel.
- Hoàn tất tất cả các mục có comment `YOUR CODE HERE`
- Chạy toàn bộ các block code để kiểm tra: Ở thanh menu, chọn Cell$\rightarrow$Run All.

---

# MẠNG CONVOLUTIONAL NEURAL NETWORK (CNN)

Bài tập này, ta sẽ tiến hành cài đặt một trong những mạng `CNN` đầu tiên là LeNet. Kiến trúc của mạng LeNet được tóm tắt như sau:

![](LeNet.png)

Trong hình trên kiến trúc của mô hình được tóm tắt như sau:

**Lớp input**: chứa ảnh đầu vào 
- Kích thước ảnh của tập MNIST là `28x28`. Do ảnh này là ảnh mức xám (không có màu) nên độ sâu bằng 1.

**Lớp convolution C1**: lớp biến đổi đặc trưng đầu tiên, sử dụng phép biến đổi convolution kết hợp với hàm kích hoạt. 

- Phiên bản năm 1998 Yann Lecun sử dụng `6 filter` kích thước `3x3`, hàm kích hoạt `sigmoid`.

**Lớp pooling** hay subsampling S2: lớp làm giảm kích thước chiều không gian (chiều ngang và dọc). 
- Phiên bản năm 1998 sử dụng filter `AveragePooling` với kích thước à `2x2`.

**Lớp convolution C3**: tương tự C1 nhưng số filter nhiều hơn *nhằm tạo ra các **đặc trưng cấp cao** kết hợp từ các đặc trưng cấp thấp trước đó*. 
- Lớp này sử dụng `16 filter` kích thước `3x3` với hàm kích hoạt `sigmoid`.

**Lớp convolution S4**: tương tự như lớp S2. 
- Ta sử dụng filter `AveragePooling`, kích thước filter là `2x2`.

**Lớp fully connected (kết nối đầu đủ) F5**: thực hiện phép nhân tuyến tính kết hợp với hàm kích hoạt. Các lớp F5 trở đi thực hiện chức năng phân loại (classify) đặc trưng đã rút trích từ các bước trước đó.
- Phiên bản này sử dụng hàm `sigmoid`. Số neuron output của lớp này là **120 neuron**.
- Lưu ý quan trọng: trước lớp F5, ta sẽ có "bước đệm" là `Flatten` để biến đặc trưng **từ dạng tensor thành dạng vector**.

**Lớp fully connected F6**: tương tự lớp F5.
- Số neuron output của lớp này là **84 neuron**, hàm kích hoạt `sigmoid`.

**Lớp output**: Tương tự như F5.
- Số neuron output của lớp này là **10 neuron**, sử dụng hàm `softmax` để đưa đặc trưng về không gian xác suất của các lớp đối tượng cần phân loại.

Bắt đầu tiến hành cài đặt thôi!

## 1. Chuẩn bị dữ liệu huấn luyện
Đầu tiên, ta sẽ sử dụng tập dữ liệu MNIST cho bài tập này. Trong `keras.datasets` có rất nhiều dữ liệu được đóng gói và chuẩn bị sẵn như `MNIST`, `CIFAR`, `Fashion MNIST`. Trong bài tập này, ta sẽ sử dụng tập dữ liệu `MNIST`:

In [ ]:
from keras.datasets import mnist
import numpy as np
from numpy.random import seed
seed(1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("Thông tin kích thước các biến trên: ")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
print('Ý nghĩa kích thước các biến trên: ')
print('Số mẫu train: {} mẫu'.format(x_train.shape[0]))
print('Số mẫu test: {} mẫu'.format(x_test.shape[0]))
print('Kích thước ảnh đầu vào: ', x_train.shape[1:])

In [ ]:
# Code block này hiển thị một mẫu dữ liệu train. 
# Đây là việc NÊN LÀM khi bắt đầu với bất kỳ dữ liệu nào
# Ở đây ta sẽ load mẫu dữ liệu thứ 1234. Các bạn có thể thay đổi giá trị này tùy ý
import matplotlib.pyplot as plt

sample_id = 1234
plt.imshow(x_train[sample_id])
plt.show()
print('Nhãn (label) của ground-truth: ', y_train[sample_id])

## 2. Tiền xử lý dữ liệu
Việc chuẩn hóa dữ liệu bao gồm hai nhóm công việc chính:
* Đối với output `y`: chuyển đổi **từ dạng nhãn (label) sang dạng one-hot encoding**.
* Đối với input `x`: chuẩn hóa các giá trị mức sáng **từ [0-255] về đoạn [0-1]**. Bước này cải tiến tốc độ huấn luyện một cách đáng kể.

In [ ]:
# Hàm này được sử dụng để chuyển đổi output từ dạng 
# nhãn sang dạng one-hot vector
def onehot(y):
    oh = np.zeros((y.shape[0], 10))
    for i in range(y.shape[0]):
        oh[i, int(y[i])]=1
    return oh

In [ ]:
from sklearn import preprocessing
import numpy as np

# Chuyển đổi từ dạng label sang dạng one-hot encoding
y_train_oh = onehot(y_train)
y_train_oh.shape

y_test_oh = onehot(y_test)
y_test_oh.shape

# Chuẩn hóa ảnh đầu vào về [0-1]
x_train_norm = x_train / 255.0
x_test_norm = x_test / 255.0

In [ ]:
# Các bạn có thể xem dữ liệu trước và sau khi tiền xử lý
# Các bạn có thể thay đổi sample_id bằng số tùy ý
sample_id = 4321

print("`y` TRƯỚC khi biến đổi: ", y_train[sample_id])
print("`y` SAU khi biến đổi: ", y_train_oh[sample_id])

In [ ]:
print("`x` TRƯỚC khi biến đổi: ", x_train[sample_id])
print("`x` SAU khi biến đổi: ", x_train_norm[sample_id])

## 3. Cài đặt mạng CNN với kiến trúc LeNet

Import các thư viện Keras phục vụ cho việc cài đặt mô hình:

Đầu tiên, ta sẽ import các thư viện, module, hàm của `Keras`:
- `Input`: để nhận dữ liệu đầu vào cho mô hình
- `Conv2D`: biến đổi Convolution với activation
- `AveragePooling2D`: giảm kích thước đặc trưng
- `Dense`: hay còn gọi là lớp Fully Connected (Lớp kết nối đầy đủ), một thành phần quan trọng của các mô hình dựa trên mạng Neural Network.
- `Model`: đây là lớp đối tượng mô hình được sử dụng để đóng gói lớp đầu vào, đầu ra, sau đó huấn luyện với phương thức `fit` và dự đoán kết quả với `predict`.
- `load_model`: hàm sử dụng để load mô hình lên từ file

In [ ]:
from keras.layers import Input, Dense, Conv2D, AveragePooling2D, Flatten
from keras.models import load_model, Model
import tensorflow as tf
import keras

In [ ]:
class LeNetCNN:
  # constructor
  def __init__(self):
    self.model = None

  # Định nghĩa kiến trúc của mô hình
  def build(self, input_dim):
    input = Input(shape = input_dim) # input
    
    # Bước rút trích đặc trưng
    C1 = Conv2D(6, (3, 3), padding='same', activation='sigmoid')(input)
    S2 = AveragePooling2D(pool_size=(2, 2), padding="same")(C1)
    C3 = Conv2D(16, (3, 3), padding='same', activation='sigmoid')(S2)
    S4 = AveragePooling2D(pool_size=(2, 2), padding="same")(C3)
    # Bước trung gian chuyển từ tensor sang vector
    flat = Flatten()(S4)
    # Bước phân loại đặc trưng bước trước
    F5 = Dense(120, activation='sigmoid', use_bias=True)(flat)
    F6 = Dense(84, activation='sigmoid', use_bias=True)(F5)
    output = Dense(10, activation='softmax', use_bias=True)(F6) # y~ output
    
    self.model = Model(input, output)

  # Huấn luyện mô hình
  def train(self, x_train, y_train): # x_train chinh la X, y_train chinh la ground-truth
    adam = keras.optimizers.Adam(learning_rate=0.01)
    self.model.compile(optimizer=adam, loss = 'categorical_crossentropy', metrics=['accuracy'])
    self.model.fit(x_train, y_train, epochs = 10, batch_size = 128)

  # Load mô hình từ file
  def load(self, model_file):
    self.model = load_model(model_file)

  # Lưu mô hình hiện tại xuống file
  def save(self, model_file):
    self.model.save(model_file)

  # Tóm tắt kiến trúc mạng
  def summary(self):
    self.model.summary()

  # Thử nghiệm mô hình với dữ liệu ảnh đầu vào
  def predict(self, x_test):
    return self.model.predict(x_test)
  
  # Đánh giá trên tập dữ liệu test
  def predict_and_eval(self, x_test_norm, y_test):
    # Predict trên tập dữ liệu test
    test_pred = self.model.predict(x_test_norm)

    # Đánh giá độ chính xác trên toàn bộ tập test
    accuracy = tf.keras.metrics.Accuracy()
    labels_pred = [np.argmax(i) for i in test_pred]
    accuracy.update_state(labels_pred, y_test)
    print('Độ chính xác của phiên bản hiện tại: {}'.format(accuracy.result().numpy()*100))


In [ ]:
# Khởi tạo đối tượng mạng CNN kiến trúc LeNet
lenet = LeNetCNN()
# Xây dựng kiến trúc mô hình với ảnh đầu vào kích thước 28x28x1
lenet.build((28, 28, 1))
lenet.summary()
lenet.train(x_train_norm, y_train_oh)

Với kiến trúc mạng ở trên, hãy quan sát và xem có thông tin nào bạn chưa thực sự hiểu rõ không?

Các thông tin cần nắm vững:
- Tên lớp
- Kích thước lớp
- Số lượng tham số

Thảo luận với bạn bè / thầy cô khi có chỗ nào chưa rõ trong các tham số trên.

In [ ]:
# Predict trên tập dữ liệu test
pred = lenet.predict(x_test_norm)
test_index = 1234
plt.imshow(x_test[test_index])
plt.show()
print('Nhãn dự đoán: ', np.argmax(pred[test_index]))
print('Nhãn ground-truth: ', y_test[test_index])

In [ ]:
# Đánh giá hiệu quả của hệ thống trên tập test
lenet.predict_and_eval(x_test_norm, y_test)

## 4. Thử nghiệm với một số biến thể

### 4.1 Biến thể Alpha - Thay activation `sigmoid` bằng `relu`

In [ ]:
class AlphaCNN(LeNetCNN):
  # Define structure of the CNN
  def build(self, input_dim):
    input = None
    output = None
    # YOUR CODE HERE
    raise NotImplementedError()
    self.model = Model(input, output)


In [ ]:
# Khởi tạo đối tượng mạng CNN kiến trúc Alpha
alpha = AlphaCNN()
# Xây dựng kiến trúc mô hình với ảnh đầu vào kích thước 28x28x1
alpha.build((28, 28, 1))
alpha.summary()

In [ ]:
# Kiểm tra kiến trúc Alpha

# Kiểm tra số layer không đổi
assert len(alpha.model.layers) == 9
# Kiểm tra một số lớp trong kiến trúc mạng Alpha
assert str(type(alpha.model.layers[0]))== "<class 'keras.engine.input_layer.InputLayer'>"
assert str(type(alpha.model.layers[1]))== "<class 'keras.layers.convolutional.conv2d.Conv2D'>"
assert str(type(alpha.model.layers[2]))== "<class 'keras.layers.pooling.average_pooling2d.AveragePooling2D'>"
assert str(type(alpha.model.layers[6]))== "<class 'keras.layers.core.dense.Dense'>"
# Kiểm tra hàm activation của lớp convolution
assert alpha.model.layers[1].activation(1.0).numpy() == 1.0
assert alpha.model.layers[1].activation(-1.0).numpy() == 0.0

In [ ]:
# Kiểm tra với bộ test ẩn

In [ ]:
# Sau khi đã đảm bảo rằng kiến trúc của bạn đã ổn. Chờ gì nữa, train thôi!
alpha.train(x_train_norm, y_train_oh)

In [ ]:
# Đánh giá hiệu quả của mô hình trên tập test
alpha.predict_and_eval(x_test_norm, y_test)

Theo bạn giữa phiên bản LeNet, Alpha, bạn thấy kết quả có sự thay đổi đáng kể hay không?

In [ ]:
options = ['Không có ý kiến', 'LeNet tốt hơn Alpha', 'LeNet tệ hơn Alpha', 'LeNet ngang ngửa Alpha (chênh không quá 1%)']
your_choice = None

# Hãy chọn your_choice bằng 0, 1, 2 hoặc 3 tương ứng với các lựa chọn ở trên
# YOUR CODE HERE
raise NotImplementedError()

print("Theo mình thì: ", options[your_choice])

### 4.2. Biến thể Beta - Thay AveragePooling bằng MaxPooling
Lưu ý rằng, biến thể `Beta` được phát triển tiếp từ biến thể `Alpha`, do đó cấu hình liên quan đến hàm activation cũng sử dụng `ReLU`

In [ ]:
from keras.layers import MaxPooling2D
class BetaCNN(LeNetCNN):
  # Define structure of the CNN
  def build(self, input_dim):
    input = None
    output = None
    # YOUR CODE HERE
    raise NotImplementedError()
    self.model = Model(input, output)


In [ ]:
# Khởi tạo đối tượng mạng CNN kiến trúc Beta
beta = BetaCNN()
# Xây dựng kiến trúc mô hình với ảnh đầu vào kích thước 28x28x1
beta.build((28, 28, 1))
beta.summary()

In [ ]:
# Kiểm tra kiến trúc Beta

# Kiểm tra số layer không đổi
assert len(beta.model.layers) == 9
# Kiểm tra một số lớp trong kiến trúc mạng
assert str(type(beta.model.layers[0]))== "<class 'keras.engine.input_layer.InputLayer'>"
assert str(type(beta.model.layers[1]))== "<class 'keras.layers.convolutional.conv2d.Conv2D'>"
assert str(type(beta.model.layers[2]))== "<class 'keras.layers.pooling.max_pooling2d.MaxPooling2D'>"
assert str(type(beta.model.layers[6]))== "<class 'keras.layers.core.dense.Dense'>"
# Kiểm tra hàm activation của lớp convolution
assert beta.model.layers[1].activation(1.0).numpy() == 1.0
assert beta.model.layers[1].activation(-1.0).numpy() == 0.0

In [ ]:
# Kiểm tra với bộ test ẩn

In [ ]:
# Train đi, chờ chi!
beta.train(x_train_norm, y_train_oh)

In [ ]:
# Đánh giá trên tập dữ liệu test
beta.predict_and_eval(x_test_norm, y_test)

Theo bạn giữa phiên bản Alpha và Beta, bạn thấy kết quả có sự thay đổi đáng kể hay không?

In [ ]:
options = ['Không có ý kiến', 'Alpha tốt hơn Beta', 'Alpha tệ hơn Beta', 'Alpha ngang ngửa Beta (chênh không quá 1%)']

your_choice = None

# Hãy chọn your_choice bằng 0, 1, 2 hoặc 3 tương ứng với các lựa chọn ở trên
# YOUR CODE HERE
raise NotImplementedError()

print("Theo tôi thì: ", options[your_choice])

Như vậy, đến bước này, bạn đã cài đặt xong mô hình LeNet theo hướng hiện đại, hay còn gọi là LeNet phiên bản 2020.

Không dừng lại ở đó, ta sẽ thử với một số biến thể táo bạo hơn.

### 4.3 Biến thể Gamma - Bỏ hết hàm activation trong biến thể Beta
Biến thể Gamma ra đời vì sự nghi ngờ rằng, **hàm activation không thực sự hữu ích** trong kiến trúc mạng CNN. Bạn hãy thử cài đặt, đánh giá xem giả thuyết này có đúng không?

In [ ]:
class GammaCNN(LeNetCNN):
  # Define structure of the CNN
  def build(self, input_dim):
    input = None
    output = None
    # YOUR CODE HERE
    raise NotImplementedError()

    self.model = Model(input, output)


In [ ]:
# Khởi tạo đối tượng mạng CNN kiến trúc Gamma
gamma = GammaCNN()
# Xây dựng kiến trúc mô hình với ảnh đầu vào kích thước 28x28x1
gamma.build((28, 28, 1))
gamma.summary()

In [ ]:
# Kiểm tra kiến trúc Gamma

# Kiểm tra số layer không đổi
assert len(gamma.model.layers) == 9
# Kiểm tra một số lớp trong kiến trúc mạng 
assert str(type(gamma.model.layers[0]))== "<class 'keras.engine.input_layer.InputLayer'>"
assert str(type(gamma.model.layers[1]))== "<class 'keras.layers.convolutional.conv2d.Conv2D'>"
assert str(type(gamma.model.layers[2]))== "<class 'keras.layers.pooling.max_pooling2d.MaxPooling2D'>"
assert str(type(gamma.model.layers[6]))== "<class 'keras.layers.core.dense.Dense'>"
# Kiểm tra hàm activation của lớp convolution
assert gamma.model.layers[1].activation(1.0) == 1.0
assert gamma.model.layers[1].activation(-1.0) == -1.0

In [ ]:
# Kiểm tra với bộ test ẩn

In [ ]:
# Train train train!
gamma.train(x_train_norm, y_train_oh)

In [ ]:
# Đánh giá trên tập dữ liệu test
gamma.predict_and_eval(x_test_norm, y_test)

Theo bạn giữa phiên bản Beta và Gamma, bạn thấy kết quả có sự thay đổi đáng kể hay không?

In [ ]:
options = ['Không có ý kiến', 'Beta tốt hơn Gamma', 'Beta tệ hơn Gamma', 'Beta ngang ngửa Gamma (chênh không quá 1%)']

your_choice = None

# Hãy chọn your_choice bằng 0, 1, 2 hoặc 3 tương ứng với các lựa chọn ở trên
# YOUR CODE HERE
raise NotImplementedError()

print("Theo tôi thì: ", options[your_choice])

### 4.4 Biến thể Omicron - Bỏ hết hàm Pooling trong biến thể Beta
Biến thể **Omicron** ra đời vì sự nghi ngờ rằng, **Bước MaxPooling không thực sự hữu ích** trong kiến trúc mạng CNN. Bạn hãy thử cài đặt, đánh giá xem giả thuyết này có đúng không?

In [ ]:
class OmicronCNN(LeNetCNN):
  # Define structure of the CNN
  def build(self, input_dim):
    input = None
    output = None
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    self.model = Model(input, output)

In [ ]:
# Khởi tạo đối tượng mạng CNN kiến trúc Omicron
omicron = OmicronCNN()
# Xây dựng kiến trúc mô hình với ảnh đầu vào kích thước 28x28x1
omicron.build((28, 28, 1))
omicron.summary()

In [ ]:
# Kiểm tra kiến trúc Omicron

# Kiểm tra số layer không đổi
assert len(omicron.model.layers) == 7
# Kiểm tra một số lớp trong kiến trúc mạng
assert str(type(omicron.model.layers[0]))== "<class 'keras.engine.input_layer.InputLayer'>"
assert str(type(omicron.model.layers[1]))== "<class 'keras.layers.convolutional.conv2d.Conv2D'>"
assert str(type(omicron.model.layers[2]))== "<class 'keras.layers.convolutional.conv2d.Conv2D'>"
assert str(type(omicron.model.layers[6]))== "<class 'keras.layers.core.dense.Dense'>"
# Kiểm tra hàm activation của lớp convolution
assert beta.model.layers[1].activation(1.0).numpy() == 1.0
assert beta.model.layers[1].activation(-1.0).numpy() == 0.0

In [ ]:
# Kiểm tra với bộ test ẩn

In [ ]:
# Train train train!
omicron.train(x_train_norm, y_train_oh)

In [ ]:
# Đánh giá trên tập dữ liệu test
omicron.predict_and_eval(x_test_norm, y_test)

Theo bạn giữa phiên bản Beta và Omicron, bạn thấy kết quả có sự thay đổi đáng kể hay không?

In [ ]:
options = ['Không có ý kiến', 'Beta tốt hơn Omicron', 'Beta tệ hơn Omicron', 'Beta ngang ngửa Omicron (chênh không quá 1%)']

your_choice = None

# Hãy chọn your_choice bằng 0, 1, 2 hoặc 3 tương ứng với các lựa chọn ở trên
# YOUR CODE HERE
raise NotImplementedError()

print("Theo tôi thì: ", options[your_choice])

Ngoài ra, bạn có nhận xét gì về số lượng tham số và thời gian train của mô hình Omicron?

In [ ]:
options = ['Omicron có tham số ít hơn Beta', 'Omicron nhiều tham số hơn tốt hơn Beta', 'Omicron có tham số bằng Beta',
           'Omicron train nhanh hơn Beta', 'Omicron train chậm hơn Beta', 'Omicron train tương đương hoặc nhanh hơn Beta']

your_choices = [1, 4]

# Hãy chọn your_choice với một tập các giá trị từ 0, 1, 2, 3, 4, 5 tương ứng với các lựa chọn ở trên
# YOUR CODE HERE
raise NotImplementedError()

print("Theo tôi thì: ")
for op in your_choices:
    print('   - ', options[op])

### 4.5 Biến thể Delta - Bỏ hết lớp Convolution trong biến thể Beta
Biến thể **Delta** ra đời vì sự nghi ngờ rằng, **Bước Convolution không thực sự hữu ích!!!??** trong kiến trúc mạng CNN. Bạn hãy thử cài đặt, đánh giá xem giả thuyết này có đúng không?

In [ ]:
class DeltaCNN(LeNetCNN):
  # Define structure of the CNN
  def build(self, input_dim):
    input = None
    output = None
    # YOUR CODE HERE
    raise NotImplementedError()
    
    self.model = Model(input, output)

In [ ]:
# Khởi tạo đối tượng mạng CNN kiến trúc Omicron
delta = DeltaCNN()
# Xây dựng kiến trúc mô hình với ảnh đầu vào kích thước 28x28x1
delta.build((28, 28, 1))
delta.summary()

In [ ]:
# Kiểm tra kiến trúc Delta

# Kiểm tra số layer không đổi
assert len(delta.model.layers) == 7
# Kiểm tra một số lớp trong kiến trúc mạng
assert str(type(delta.model.layers[0]))== "<class 'keras.engine.input_layer.InputLayer'>"
assert str(type(delta.model.layers[1]))== "<class 'keras.layers.pooling.max_pooling2d.MaxPooling2D'>"
assert str(type(delta.model.layers[2]))== "<class 'keras.layers.pooling.max_pooling2d.MaxPooling2D'>"
assert str(type(delta.model.layers[6]))== "<class 'keras.layers.core.dense.Dense'>"
# Kiểm tra hàm activation của lớp convolution
assert delta.model.layers[1].pool_size  == (2, 2)

In [ ]:
# Kiểm tra với bộ test ẩn

In [ ]:
# Trèn trén tren!
delta.train(x_train_norm, y_train_oh)

In [ ]:
# Đánh giá trên tập dữ liệu test
delta.predict_and_eval(x_test_norm, y_test)

Theo bạn giữa phiên bản Delta và Beta, bạn thấy kết quả có sự thay đổi đáng kể hay không?

In [ ]:
options = ['Không có ý kiến', 'Beta tốt hơn Delta', 'Beta tệ hơn Delta', 'Beta ngang ngửa Delta (chênh không quá 1%)']

your_choice = None

# Hãy chọn your_choice bằng 0, 1, 2 hoặc 3 tương ứng với các lựa chọn ở trên
# YOUR CODE HERE
raise NotImplementedError()

print("Theo tôi thì: ", options[your_choice])

Ngoài ra, bạn có nhận xét gì về số lượng tham số và thời gian train của mô hình Delta?

In [ ]:
options = ['Delta có tham số ít hơn Beta', 'Delta nhiều tham số hơn tốt hơn Beta', 'Delta có tham số bằng Beta',
           'Delta train nhanh hơn Beta', 'Delta train chậm hơn Beta', 'Delta train tương đương hoặc nhanh hơn Beta']

your_choices = []

# Hãy chọn your_choice với một tập các giá trị từ 0, 1, 2, 3, 4, 5 tương ứng với các lựa chọn ở trên
# YOUR CODE HERE
raise NotImplementedError()

print("Theo tôi thì: ")
for op in your_choices:
    print('   - ', options[op])

Phew!! Vậy là bạn đã trải nghiệm cảm giác thiết kế, chỉnh sửa kiến trục mạng CNN. Mỗi module đều có một chức năng nhất định. Hi vọng bạn cảm nhận được ý nghĩa của từng module và vai trò của nó trong thiết kế chung của mạng CNN.